\begin{align*}
z^{[1]} & = W^{[1]} x + W_0^{[1]} \\
h & = \sigma (z^{[1]}) \\
z^{[2]} & = W^{[2]} h + W_0^{[2]} \\
o & = \sigma (z^{[2]}) \\
L & = \frac{1}{m} \sum_{i = 1}^{m} (o^{(i)} - y^{(i)})^2 = \frac{1}{m} (o - y)^T (o - y) \\
\end{align*}

Using chain rule and tensor denotation one can get:
\begin{align*}
\frac{\partial L}{\partial W^{[L-1]}} = (\sigma^\prime(z^{[L]}) \odot \frac{\partial L}{\partial o})_\nu \sigma^\prime (z_{i\nu}^{[L-1]})a_{j \nu}^{[L-2]} w_i^{[L]} = (W^{[L]} \odot \sigma^\prime(z^{[L-1]})) (\sigma^\prime(z^{[L]}) \odot \frac{\partial L}{\partial o} \odot a^{[L-2]})
\end{align*}

Then for $\frac{\partial L}{\partial w^{1}_{12}}$:

\begin{align*}
\frac{\partial L}{\partial w^{1}_{12}} = \frac{\partial L}{\partial W^{1}_{21}} = \frac{2}{m} w_{2}^2 (h_2 \odot (1 - h_2)) o \odot (1-o) \odot (o-y)\odot x
\end{align*}

In case of step activation function (not linear) it is possible to achieve 100% accuracy (optimal weights are provided in src.p01_nn.py). Basically, you construct the triangle (3 neurons for 3 sides). 

Using linear activation function the $o$ will be linear combination of the inputs $x$. So, the decision boundary will be a straight line for any weights. Thus, 100% accuracy is not possible. 